In [1]:
require 'nn'
require 'hdf5'
require 'optim'

In [2]:
myFile = hdf5.open('5-grams.hdf5','r')
data = myFile:all()
myFile:close()

In [3]:
data

{
  train_nocounts : DoubleTensor - size: 887522x5
  train_1000 : DoubleTensor - size: 887522x5
  test : LongTensor - size: 3761x54
  train : LongTensor - size: 754037x6
  nwords : LongTensor - size: 1
  valid : LongTensor - size: 3370x54
  train_1000_nocounts : LongTensor - size: 620208x6
  valid_output : LongTensor - size: 3370x50
}


In [48]:
nwords = 10001
train = data['train_nocounts']:narrow(2,1,5)
train_input = train:narrow(2,1,4)
train_output = train:narrow(2,5,1)

valid_topredict = data['valid']:narrow(2,1,50)
valid_input = data['valid']:narrow(2,51,4)
valid_output = data['valid_output']

In [53]:
dataset={};
for i=1,train:size(1) do 
  dataset[i] = {train_input[i]:view(1,4), train_output[i]}
end
function dataset:size() return train:size(1) end

In [54]:
N = 5
dwin = N-1
hid1 = 25.
hid2 = 80
dnnlm = nn.Sequential()

LT = nn.Sequential()
LT_ = nn.LookupTable(nwords,hid1)
LT:add(LT_)
LT:add(nn.View(1,-1,hid1*dwin))
LT:add(nn.Squeeze()) 

dnnlm:add(LT)

concat = nn.ConcatTable()

lin_tanh = nn.Sequential()
lin_tanh:add(nn.Linear(hid1*dwin,hid2))
lin_tanh:add(nn.Tanh())

id = nn.Identity()

concat:add(lin_tanh)
concat:add(id)

dnnlm:add(concat)
dnnlm:add(nn.JoinTable(1))
dnnlm:add(nn.Linear(hid2+hid1*dwin, nwords))
dnnlm:add(nn.LogSoftMax())

In [55]:
criterion = nn.ClassNLLCriterion()

In [51]:
kag_pred_valid = torch.Tensor(valid_input:size(1),50)
norm_mat = torch.zeros(valid_input:size(1),50)

In [57]:
for i = 1, 1 do
    
    timer = torch.Timer()
    
    LT_.weight:renorm(2,1,1)
    
    trainer = nn.StochasticGradient(dnnlm, criterion)
    trainer.learningRate = 0.005
    trainer.maxIteration = 1
    trainer:train(dataset)
    print('Epoch '..i..': '..timer:time().real)
    
    -- Evaluatin perplexity on validation:
    kag_pred_valid:zero()

    for i = 1, valid_input:size(1) do
        kag_pred_valid[i]:copy(dnnlm:forward(valid_input[i]):index(1, valid_topredict[i])):exp()
    end
    
    norm_mat:zero()
    norm_mat:copy(torch.expandAs(kag_pred_valid:sum(2), kag_pred_valid))
    kag_pred_valid:cdiv(norm_mat)
    
    CE = 0
    for i = 1, valid_input:size(1) do
        mm,aa = valid_output[i]:max(1)
        CE = CE + math.log(kag_pred_valid[i][aa[1]])
    end
    
    val_res = math.exp(-CE/kag_pred_valid:size(1))
    print('Perplexity on valid: '..val_res)
    
end

# StochasticGradient: training	


# current error = 6.0318624286766	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 6.0318624286766	
Epoch 1: 3777.7758259773	


Perplexity on valid: 5.8055104706363	


In [58]:
for i = 2, 9 do
    
    timer = torch.Timer()
    
    LT_.weight:renorm(2,1,1)
    
    trainer = nn.StochasticGradient(dnnlm, criterion)
    trainer.learningRate = 0.005
    trainer.maxIteration = 1
    trainer:train(dataset)
    print('Epoch '..i..': '..timer:time().real)
    
    -- Evaluatin perplexity on validation:
    kag_pred_valid:zero()

    for i = 1, valid_input:size(1) do
        kag_pred_valid[i]:copy(dnnlm:forward(valid_input[i]):index(1, valid_topredict[i])):exp()
    end
    
    norm_mat:zero()
    norm_mat:copy(torch.expandAs(kag_pred_valid:sum(2), kag_pred_valid))
    kag_pred_valid:cdiv(norm_mat)
    
    CE = 0
    for i = 1, valid_input:size(1) do
        mm,aa = valid_output[i]:max(1)
        CE = CE + math.log(kag_pred_valid[i][aa[1]])
    end
    
    val_res = math.exp(-CE/kag_pred_valid:size(1))
    print('Perplexity on valid: '..val_res)
    
end

# StochasticGradient: training	


# current error = 5.5262946470065	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 5.5262946470065	
Epoch 2: 3650.7690629959	


Perplexity on valid: 5.367000690273	


# StochasticGradient: training	


# current error = 5.3135841631288	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 5.3135841631288	
Epoch 3: 3620.9383718967	


...

# current error = 4.8692454295903	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 4.8692454295903	
Epoch 8: 3622.6675741673	


Perplexity on valid: 5.0872367119564	


# StochasticGradient: training	


# current error = 4.8223222568596	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 4.8223222568596	
Epoch 9: 3623.1487159729	


Perplexity on valid: 5.0862021600192	
